# Task 2
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [72]:
import numpy as np
import pandas as pd
# Add any other imports you need here
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, RBF, Matern, RationalQuadratic
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [73]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/My Drive/Colab Notebooks/IML/Task 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/IML/Task 2


# Data Loading
TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test
(and potentially change initialization of variables to accomodate how you deal with non-numeric data)

In [74]:
"""
This loads the training and test data, preprocesses it, removes the NaN
values and interpolates the missing data using imputation

Parameters
----------
Compute
----------
X_train: matrix of floats, training input with features
y_train: array of floats, training output with labels
X_test: matrix of floats: dim = (100, ?), test input with features
"""
# Load training data
train_df = pd.read_csv("train.csv")
columns_train = train_df.columns.tolist() # Will be needed later

print("Training data:")
print("Shape:", train_df.shape)
print("Column names:", columns_train)
print(train_df.head(2))
print('\n')

# Load test data
test_df = pd.read_csv("test.csv")
columns_test = test_df.columns.tolist() # Will be needed later

print("Test data:")
print("Shape:", test_df.shape)
print("Column names:", columns_test)
print(test_df.head(2))

Training data:
Shape: (900, 11)
Column names: ['season', 'price_AUS', 'price_CHF', 'price_CZE', 'price_GER', 'price_ESP', 'price_FRA', 'price_UK', 'price_ITA', 'price_POL', 'price_SVK']
   season  price_AUS  price_CHF  price_CZE  price_GER  price_ESP  price_FRA  \
0  spring        NaN   9.644028  -1.686248  -1.748076  -3.666005        NaN   
1  summer        NaN   7.246061  -2.132377  -2.054363  -3.295697  -4.104759   

   price_UK  price_ITA  price_POL  price_SVK  
0 -1.822720  -3.931031        NaN  -3.238197  
1 -1.826021        NaN        NaN  -3.212894  


Test data:
Shape: (100, 10)
Column names: ['season', 'price_AUS', 'price_CZE', 'price_GER', 'price_ESP', 'price_FRA', 'price_UK', 'price_ITA', 'price_POL', 'price_SVK']
   season  price_AUS  price_CZE  price_GER  price_ESP  price_FRA  price_UK  \
0  spring        NaN   0.472985   0.707957        NaN  -1.136441 -0.596703   
1  summer  -1.184837   0.358019        NaN  -3.199028  -1.069695       NaN   

   price_ITA  price_POL  pric

In [75]:
# TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test

def preprocess(train_df, test_df, imputer):
    # 1. Impute missing values (features) in the train and test datasets using Imputer
    X_train_season = train_df["season"].to_numpy()
    Y_train = train_df["price_CHF"].to_numpy()
    X_train_other_features = train_df.drop(["season", "price_CHF"], axis=1).to_numpy()

    X_test_season = test_df["season"].to_numpy()
    X_test_other_features = test_df.drop("season", axis=1).to_numpy()

    imputer.fit(X_train_other_features)
    X_train_transformed = imputer.transform(X_train_other_features)
    X_test_transformed = imputer.transform(X_test_other_features)

    # 2. Remove samples with missing labels from the train dataset
    not_nan_indexes = np.where(~np.isnan(Y_train))[0]
    X_train_transformed = X_train_transformed[not_nan_indexes]
    X_train_season = X_train_season[not_nan_indexes]
    Y_train = Y_train[not_nan_indexes] # Ready to be used in the regression

    # 3. Perform OneHotEncoding to handle a categorical non-numeric feature "season"
    one_hot_encoder = OneHotEncoder(drop='first') # drop='first' is to prevent multicollinearity
    X_train_season_tranformed = one_hot_encoder.fit_transform(X_train_season.reshape(-1, 1)).toarray()
    X_test_season_tranformed = one_hot_encoder.fit_transform(X_test_season.reshape(-1, 1)).toarray()

    # 4. Create final datasets to be used in the regression
    X_train = np.concatenate((X_train_season_tranformed, X_train_transformed), axis=1)
    X_test = np.concatenate((X_test_season_tranformed, X_test_transformed), axis=1)

    return X_train, Y_train, X_test

# Modeling and Prediction
TODO: Define the model and fit it using training data. Then, use test data to make predictions

In [76]:
#TODO: Define the model and fit it using training data. Then, use test data to make predictions

In [77]:
# Function to choose a kernel using cross-validation

def cross_validation(X_train, Y_train, X_test, n_folds, models):
    n_models = len(models)

    R2_mat = np.zeros((n_folds, n_models))

    k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    for model_ind in range(n_models):

      fold_ind = 0
      for train_ind, val_ind in k_fold.split(X_train):
          x_train, x_val = X_train[train_ind], X_train[val_ind]
          y_train, y_val = Y_train[train_ind], Y_train[val_ind]

          models[model_ind].fit(x_train, y_train)
          y_pred = models[model_ind].predict(x_val)
          R2_mat[fold_ind][model_ind] = r2_score(y_val, y_pred)
          fold_ind += 1

    avg_R2 = np.mean(R2_mat, axis=0)

    max_R2_ind = np.argmax(avg_R2)
    max_R2 = avg_R2[max_R2_ind]
    best_model = models[max_R2_ind]

    best_model.fit(X_train, Y_train)
    y_pred = best_model.predict(X_test)

    return max_R2, best_model, y_pred

In [78]:
gprDot = GaussianProcessRegressor(kernel=DotProduct())
gprMat = GaussianProcessRegressor(kernel=Matern())
gprRQ = GaussianProcessRegressor(kernel=RationalQuadratic())
models = [gprDot, gprMat, gprRQ]
n_folds = 5

In [82]:
# Perform cross-validation on dataset preprocessed with IterativeImputer

imputer_II = IterativeImputer()
X_train_II, Y_train_II, X_test_II = preprocess(train_df, test_df, imputer_II)

max_R2, best_model, y_pred = cross_validation(X_train_II, Y_train_II, X_test_II, n_folds, models)
print(f"Results for dataset preprocessed with IterativeImputer")
print(f"Max R2: {max_R2:.4f}")
print(f"Best model: {best_model}")

Results for dataset preprocessed with IterativeImputer
Max R2: 0.9629
Best model: GaussianProcessRegressor(kernel=RationalQuadratic(alpha=1, length_scale=1))


In [83]:
# Perform cross-validation on dataset preprocessed with KNNImputer

imputer_KNN = KNNImputer(n_neighbors=10)
X_train_KNN, Y_train_KNN, X_test_KNN = preprocess(train_df, test_df, imputer_KNN)
max_R2, best_model, y_pred = cross_validation(X_train_KNN, Y_train_KNN, X_test_KNN, n_folds, models)
print(f"Results for dataset preprocessed with KNNImputer")
print(f"Max R2: {max_R2:.4f}")
print(f"Best model: {best_model}")

Results for dataset preprocessed with KNNImputer
Max R2: 0.9800
Best model: GaussianProcessRegressor(kernel=RationalQuadratic(alpha=1, length_scale=1))


# Saving Results
You don't have to change this

In [85]:
dt = pd.DataFrame(y_pred)
dt.columns = ['price_CHF']
dt.to_csv('results.csv', index=False)
print("\nResults file successfully generated!")


Results file successfully generated!
